# Install required apt packages

In [ ]:
# @title Install required apt packages
!apt-get update
!apt-get install -y build-essential python3.11 python3.11-dev python3.11-venv unzip
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!update-alternatives --install /usr/bin/python python /usr/bin/python3.11 1
!python3 --version
!python --version
!echo "-= Done. =-"

# Environment Setup

In [ ]:
# @title Environment Setup
from pathlib import Path
import os

OPTIONS = {}

UPDATE_FLUXGYM = False  #@param {type:"boolean"}
UPDATE_KOHYA_SS = False  #@param {type:"boolean"}

OPTIONS['UPDATE_FLUXGYM'] = UPDATE_FLUXGYM
OPTIONS['UPDATE_KOHYA_SS'] = UPDATE_KOHYA_SS

APPLICATIONS_DIR="/opt/applications"

AI_WORKSPACE = f"{APPLICATIONS_DIR}/AI"
FLUXGYM_WORKSPACE = f"{APPLICATIONS_DIR}/AI/fluxgym"
KOHYA_SS_WORKSPACE = f"{APPLICATIONS_DIR}/AI/fluxgym/sd-scripts"

!echo "Creating AI workspace (if it doesn't exist)"
os.makedirs(AI_WORKSPACE, exist_ok=True)

%cd $AI_WORKSPACE

![ ! -d $FLUXGYM_WORKSPACE ] && echo -= Initial setup FLUXGYM =- && git clone https://github.com/cocktailpeanut/fluxgym $FLUXGYM_WORKSPACE

%cd $FLUXGYM_WORKSPACE

![ ! -d $KOHYA_SS_WORKSPACE ] && echo -= Initial setup Kohya_ss =- && git clone -b sd3 https://github.com/kohya-ss/sd-scripts $KOHYA_SS_WORKSPACE

if OPTIONS['UPDATE_FLUXGYM']:
  !echo "-= Updating FLUXGYM =-"
  %cd $FLUXGYM_WORKSPACE
  !git pull

if OPTIONS['UPDATE_KOHYA_SS']:
  !echo "-= Updating Kohya_ss =-"
  %cd KOHYA_SS_WORKSPACE
  !git pull

%cd $FLUXGYM_WORKSPACE    

!python -m venv env
!source env/bin/activate

%cd $KOHYA_SS_WORKSPACE

%pip install -r requirements.txt

%cd $FLUXGYM_WORKSPACE

%pip install -r requirements.txt

!pip install matplotlib-inline
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

!echo "-= Done. =-"

# Install rathole and configure

In [ ]:
# @title Install rathole and configure
import os

RATHOLE_SERVER_IP = "15.235.162.159"  #@param {type:"string"}
RATHOLE_SERVER_PORT = 2333 # @param {"type":"number"}
RATHOLE_PATH = "/opt/applications/rathole" # @param {"type":"string"}
FLUXGYM_PORT = 7860 # @param {"type":"number"}

os.makedirs(RATHOLE_PATH, exist_ok=True)

%cd $RATHOLE_PATH

!wget https://github.com/rapiz1/rathole/releases/download/v0.5.0/rathole-x86_64-unknown-linux-gnu.zip
!unzip -o rathole-x86_64-unknown-linux-gnu.zip; rm -rf rathole-x86_64-unknown-linux-gnu.zip
!chmod 775 rathole
#Create client configuration
!echo "[client]" > client.toml
!echo "remote_addr = \"$RATHOLE_SERVER_IP:$RATHOLE_SERVER_PORT\"" >> client.toml
!echo "" >> client.toml
!echo "[client.services.fluxgymtcpa]" >> client.toml
!echo "type = \"tcp\"" >> client.toml
!echo "token = \"tSoB3rhg~=\"" >> client.toml
!echo "local_addr = \"localhost:$FLUXGYM_PORT\"" >> client.toml
!echo "-= Done. =-"

# Start Rathole tunnel and FluxGym

In [ ]:
# @title Start Rathole tunnel and FluxGym
import subprocess
import threading
import time
import socket
import urllib.request

ratholepath = RATHOLE_PATH

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("Starting Rathole")
  p = subprocess.Popen([ratholepath + "/rathole", "--client", ratholepath + "/client.toml"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  !echo "-= Rathole started =-"

threading.Thread(target=iframe_thread, daemon=True, args=(FLUXGYM_PORT,)).start()

%cd $FLUXGYM_WORKSPACE

!python3 app.py
#Use modified app.py if --listen and --port are needed
#!python3 app.py --listen="0.0.0.0" --port=$FLUXGYM_PORT

!echo "-= Done. =-"

# Install cloudflare

In [ ]:
# @title Install cloudflare
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!rm -rf cloudflared-linux-amd64.deb
!echo "-= Done. =-"

# Start cloudflare tunnel and FluxGym

In [ ]:
# @title Start cloudflare tunnel and FluxGym
import subprocess
import threading
import time
import socket
import urllib.request

FLUXGYM_PORT = 7860 # @param {"type":"number"}

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nFLUXGYM finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      cf_url = l[l.find("http"):]
      print("This is the URL to access ComfyUI:", cf_url, end='')

threading.Thread(target=iframe_thread, daemon=True, args=(FLUXGYM_PORT,)).start()

%cd $FLUXGYM_WORKSPACE

!python3 app.py
#Use modified app.py if --listen and --port are needed
#!python3 app.py --listen="0.0.0.0" --port=$FLUXGYM_PORT


!echo "-= Done. =-"